This workbook writes randomly played replay-buffers to a Teradata database, and then triggers a Teradata SQL Stored Procedure to assign discounted rewards and build a table with average rewards.

In [15]:
import random
import numpy as np
import pandas as pd
import time
import chess
from teradataml import *

The block below sets the database connection details and specifies the target tables.

In [16]:
# Connect with Teradata Database
eng = create_context("192.168.56.1", "val", "val")
conn = get_connection()
replay_buffer_table = "replay_buffer"
replay_buffer_staging_table = "replay_buffer_stage"

The block below sets the Pandas DataFrame and seeds the randomizer.

In [17]:
batch_df = pd.DataFrame({
    'game_id': [],
    'ply': [],
    'FEN': [],
    'move': [],
    'reward': [],
})

random.seed(time.time())

The block below creates some helper functions.

In [18]:
def play_game(in_board, game_id, training=None):
    
    # Initialize the starting variables.
    sequence_no = 0
    reward = 0
    
    game_df = pd.DataFrame({
        'game_id': [],
        'ply': [],
        'FEN': [],
        'move': [],
        'reward': [],
    })
    board = chess.Board()
    if in_board is not None:
        board = in_board

    # Launch into the key loop.
    while not board.is_checkmate() and not board.is_stalemate() and not board.is_insufficient_material() and not board.can_claim_draw() and not board.is_seventyfive_moves() and not board.is_fivefold_repetition():
        if training and sequence_no == 0:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            opening_fen = fen[:spaces_pos[3]]
            
            ply_array = np.array([game_id, sequence_no, opening_fen, "", 0])
            new_row = pd.DataFrame([ply_array], columns=game_df.columns)
            game_df = pd.concat([game_df, new_row], ignore_index=True)

        sequence_no += 1
        all_moves = str(board.legal_moves)
        start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
        end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
        all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
        all_moves_string = all_moves_string.replace(" ", "")
        all_moves_array = all_moves_string.split(',')
        # random.seed(time.time())
        random_int = random.randint(0, len(all_moves_array)-1)
        next_move = all_moves_array[random_int]
        board.push_san(next_move)

        if training:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            trimmed_fen = fen[:spaces_pos[3]]
            

            ply_array = np.array([game_id, sequence_no, trimmed_fen, next_move, 0])
            new_row = pd.DataFrame([ply_array], columns=game_df.columns)
            game_df = pd.concat([game_df, new_row], ignore_index=True)

    if board.is_checkmate():
        if board.outcome().winner:
            reward = 1
        else:
            reward = -1
    
    reward_array = np.full(sequence_no+1, reward)
    game_df['reward'] = reward_array

    return game_df, reward

The block below orchestrates the main logic flow.  It plays a batch of games specified by num_games, writing the replay-buffers (stored in the Pandas DataFrame) to Teradata in batches of batch_size.

In [19]:
num_games = 10000    # DISTINCT FEN STATES BEFORE: 47 072 837

batch_size = 200
batch_counter = 0
out_reward = 0
start_time = time.time()

# Set the game_id by looking up the last used game_id.
game_id_cursor = execute_sql("SELECT MAX(game_id) FROM " + replay_buffer_table +";")
in_game_id = int(game_id_cursor.fetchall()[0][0]) + 1  # There has to be a better way to do this.

for counter in range(num_games):
    batch_counter += 1
    out_replay_buffer_df, out_reward = play_game(None, in_game_id, training=True)
    print("Reward for game " + str(in_game_id) + " was " + str(out_reward) + ".")
   
    # Increment the game_id
    in_game_id += 1

    # Append the game dataframe to the batch dataframe.
    batch_df = pd.concat([batch_df, out_replay_buffer_df], ignore_index=True)
    
    if batch_counter == batch_size or counter == num_games-1:
        
        # Load the batch to the database.
        fastload(df = batch_df, table_name = replay_buffer_staging_table, if_exists = 'append')
        
        # Empty the batch dataframe.
        batch_df = pd.DataFrame({
            'game_id': [],
            'ply': [],
            'FEN': [],
            'move': [],
            'reward': [],
        })
        
        # Reset the batch counter.
        batch_counter = 0
        end_time = time.time()
        batch_duration = end_time-start_time
        print(str(counter+1) + " of " + str(num_games) + " completed. Last batch duration was " + str(batch_duration) + " with average game duration of " + str(batch_duration/batch_size) + ".")
        start_time = time.time()


Reward for game 140000 was 0.
Reward for game 140001 was 0.
Reward for game 140002 was -1.
Reward for game 140003 was 0.
Reward for game 140004 was 0.
Reward for game 140005 was 0.
Reward for game 140006 was 0.
Reward for game 140007 was 0.
Reward for game 140008 was 0.
Reward for game 140009 was 0.
Reward for game 140010 was 0.
Reward for game 140011 was 0.
Reward for game 140012 was 0.
Reward for game 140013 was -1.
Reward for game 140014 was 0.
Reward for game 140015 was 0.
Reward for game 140016 was 0.
Reward for game 140017 was 0.
Reward for game 140018 was 0.
Reward for game 140019 was -1.
Reward for game 140020 was 0.
Reward for game 140021 was 0.
Reward for game 140022 was 0.
Reward for game 140023 was 0.
Reward for game 140024 was 0.
Reward for game 140025 was 0.
Reward for game 140026 was 0.
Reward for game 140027 was 0.
Reward for game 140028 was 1.
Reward for game 140029 was 0.
Reward for game 140030 was 0.
Reward for game 140031 was 0.
Reward for game 140032 was 0.
Reward 

Once the replay_buffers have been written to the Teradata database, it kicks off a Stored Procedure to update the rewards per half-move using a Discount Factor, and then build a table with aggregate rewards per board position.

In [20]:
execute_sql("CALL UpdateProcessBatch()")

TeradataCursor uRowsHandle=4534 bClosed=False

Closes and release the connection to the database.

In [21]:
conn.close
remove_context()

True